# Session 04: Model Context Protocol (MCP)

MCP (Model Context Protocol; https://modelcontextprotocol.io/docs/getting-started/intro) is an open-source standard for connecting LLMs with external systems.
It is commonly implemented as a REST server that exposes capabilities in a predefined protocol.

MCP servers can provide three main types of capabilities:

| Feature	 | Explanation	                                                                                                                                                                                                  | Examples	                |
|:-----------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------|
| Resources	 | Passive data sources that provide read-only access to information for context.  Similar to GET endpoints in a REST API - they provide data but shouldn't perform significant computation or have side effects | "Access knowledge base"	 |
| Tools	     | Functions that the LLM can actively call, and decides when to use them based on user requests. Tools are expected to perform computation and can have side effects.                                           | "Book a flight"	         |
| Prompts    | Pre-built instruction templates that tell the model to work with specific tools and resources.                                                                                                                | "Summarize my meeting"   |


In [1]:
import json
from IPython.display import display, Markdown

def mdprint(text):
    """Helper function for printing markdown text."""
    display(Markdown(text))

def pprint(result):
    """Helper function for pretty-printing raw model responses."""
    for item in result.new_items:
        print(item.__class__, json.dumps(item.to_input_item(), indent=2))

## Instantiating the Server

Use the official MCP Python SDK (`mcp`; https://github.com/modelcontextprotocol/python-sdk)  to quickly instantiate a server:

In [2]:
from mcp.server.fastmcp import FastMCP

# Create server
server = FastMCP("Weather Recommendation Bot")

## Adding Tools

Add three tools to the server:
- a tool to get the current time
- a tool to get the current location
- a tool to get the weather forecast for a given location

Adding tools is similar to how it is done in `agents`: just annotate your tool call function; use the `@server.tool()` annotation to register the tool to you `server` object.

In [3]:
import time
import datetime
import requests

@server.tool()
def get_weather(city: str) -> str:
    """Get weather forecast for a given city."""
    endpoint = "https://wttr.in"
    response = requests.get(f"{endpoint}/{city}?format=j1&2").json()
    return response

@server.tool()
def get_current_time() -> str:
    """Get current time."""
    return time.strftime("%I:%M:%S %p")

@server.tool()
def get_location() -> str:
    """Get current location."""
    endpoint = "https://ipinfo.io"
    response = requests.get(f"{endpoint}")
    return response.json()["city"]


## Adding resources

Similarly, we can add the `@server.resource(<uri>)` decorate to register resources. Implement a resource that returns the users clothing inventory (use mock data).

*Note*: this might not be supported for all models and APIs; if you face issues with models using it, register it as a tool instead.

In [4]:
mock_data = [
    ("jackets", "wool jacket", "charcoal gray", 5),
    ("jackets", "rain jacket", "navy blue", 4),
    ("jackets", "winter coat", "forest green", 4),
    ("jackets", "blazer", "midnight blue", 2),
    ("jackets", "denim jacket", "faded blue", 3),
    ("jackets", "windbreaker", "bright red", 4),
    ("jackets", "leather jacket", "black", 3),
    ("jackets", "puffer jacket", "olive green", 5),

    ("upper_layers", "t-shirt (short sleeve)", "white", 5),
    ("upper_layers", "t-shirt (long sleeve)", "heather gray", 4),
    ("upper_layers", "polo shirt", "cobalt blue", 3),
    ("upper_layers", "flannel shirt", "red plaid", 4),
    ("upper_layers", "dress shirt", "light blue", 2),
    ("upper_layers", "hoodie (lightweight)", "charcoal", 5),
    ("upper_layers", "hoodie (heavyweight)", "dark green", 4),
    ("upper_layers", "sweater (thin)", "cream", 4),
    ("upper_layers", "sweater (thick wool)", "oatmeal", 5),
    ("upper_layers", "cardigan", "burgundy", 3),
    ("upper_layers", "thermal base layer", "black", 4),

    ("lower_layers", "jeans (regular)", "indigo", 5),
    ("lower_layers", "jeans (thermal lined)", "dark wash", 4),
    ("lower_layers", "chinos", "khaki", 3),
    ("lower_layers", "cargo pants", "olive drab", 4),
    ("lower_layers", "shorts", "light gray", 5),
    ("lower_layers", "sweatpants", "charcoal", 4),
    ("lower_layers", "dress pants", "navy", 2),
    ("lower_layers", "leggings/base layer", "black", 3),

    ("footwear", "sneakers", "white", 5),
    ("footwear", "running shoes", "neon green", 4),
    ("footwear", "boots (waterproof)", "brown", 5),
    ("footwear", "boots (winter insulated)", "black", 4),
    ("footwear", "sandals", "tan", 3),
    ("footwear", "dress shoes", "oxblood", 2),
    ("footwear", "hiking boots", "dark brown", 4),

    ("accessories", "umbrella", "black", 5),
    ("accessories", "sunglasses", "matte black", 4),
    ("accessories", "baseball cap", "navy", 4),
    ("accessories", "beanie", "charcoal", 5),
    ("accessories", "sun hat (wide brim)", "beige", 3),
    ("accessories", "scarf (light)", "sky blue", 3),
    ("accessories", "scarf (wool)", "dark gray", 4),
    ("accessories", "gloves (light)", "black", 3),
    ("accessories", "gloves (winter insulated)", "navy", 4),
    ("accessories", "mittens", "red", 3),
    ("accessories", "neck gaiter", "dark green", 4),

    ("specialized", "rain pants", "black", 3),
    ("specialized", "snow pants", "charcoal", 4),
    ("specialized", "vest (puffer)", "navy", 4),
    ("specialized", "vest (fleece)", "gray", 3),
    ("specialized", "rain poncho", "yellow", 2),
    ("specialized", "ear muffs", "black", 3),
    ("specialized", "face mask (cold weather)", "dark gray", 4),
]


In [5]:
#@server.resource("clothing://inventory") # Note: this is not supported by gpt-OSS via the API; use a tool call instead.
@server.tool()
def get_clothing_inventory() -> list[tuple[str, str, str, int]]:
    """Get the clothing inventory of the user. Returns tuples of (category, item, color, preference rating)"""
    return mock_data

## Adding Prompts

Prompts are essentially server-side functions to transform structure *input*  into a natural language prompt for the model. These are not called by the model directly, but instead by the user to obtain model input.

You can register a prompt template with `@server.prompt(title=...)`.

In [6]:
from typing import Literal

@server.prompt(title="Outfit Choice Assistant")
def choose_outfit(style: str, time: Literal["today", "tomorrow"]) -> str:
    """
    Generate agent instructions for weather-dependent recommendations.

    :arg str style: The desired style of the outfit.
    :arg str time: The day to plan the outfit for.
    """
    return f"""
You are a helpful weather advisor providing personalized clothing recommendations.

INSTRUCTIONS:
- Use available tools to determine current location, time and weather to determine the appropriate clothing
- Use the users clothing inventory to make concrete choices for a full outfit
- Plan the for this date: {time}
- Plan the in this style: {style}

Always base your advice on real-time weather data from the tools. Return the recommended outfit as a markdown table, with layer, item recommendation, and reasoning as columns.
"""

In [7]:
from agents import Agent, Runner, OpenAIChatCompletionsModel
from agents.mcp import MCPServer, MCPServerStreamableHttp
from openai import AsyncOpenAI
from api_key import API_KEY
API_URL = "https://api.helmholtz-blablador.fz-juelich.de/v1/"
#API_KEY = "<KEY>"
API_MODEL = "1 - GPT-OSS-120b - an open model released by OpenAI in August 2025"

In [8]:
async def run(style: str, time: Literal["today", "tomorrow"]) -> str:
    async with MCPServerStreamableHttp(params={"url": "http://127.0.0.1:8000/mcp"}) as server:
        agent = Agent(
            name="Agent",
            mcp_servers=[server],
            model=OpenAIChatCompletionsModel(
                model=API_MODEL,
                openai_client=AsyncOpenAI(api_key=API_KEY, base_url=API_URL)
            )
        )
        prompt = await server.get_prompt("choose_outfit", {"style": style, "time": time})
        out = await Runner.run(starting_agent=agent, input=prompt.messages[0].content.text)
    return out

In [10]:
out = await run(style="sporty", time="tomorrow")
mdprint(out.final_output)

[11/20/25 12:33:12] INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"         ]8;id=322725;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=807326;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 0a074077175a4044a804151befa1bb67           ]8;id=225902;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=928526;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=897573;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=493902;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py#146\146]8;;\

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=468288;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=158364;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"          ]8;id=220526;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=774034;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"         ]8;id=536463;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=623676;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

[11/20/25 12:33:19] WARNING  OPENAI_API_KEY is not set, skipping trace export                      ]8;id=616325;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/agents/tracing/processors.py\processors.py]8;;\:]8;id=401051;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/agents/tracing/processors.py#96\96]8;;\

[11/20/25 12:33:23] INFO     HTTP Request: POST                                                     ]8;id=190226;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=228810;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://api.helmholtz-blablador.fz-juelich.de/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"       ]8;id=312370;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=542583;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

**Tomorrow (2025‑11‑21) in Leipzig** – high ≈ 2 °C, low ≈ ‑2 °C, clear, no rain.  
You’ll need solid insulation and a sporty, active‑wear vibe.

**Outfit (all items you own, highest‑rated choices):**

| Layer | Item (rating) | Why it fits |
|-------|----------------|------------|
| Base | **Thermal base layer** – black (4) | Locks heat in, thin enough for mobility. |
| Mid  | **Heavyweight hoodie** – dark green (4) + **Thick wool sweater** – oatmeal (5) | Hoodie gives sport‑style; sweater adds extra warmth. |
| Outer| **Puffer jacket** – olive green (5) | Best insulation for sub‑freezing temps, still sporty. |
| Bottom| **Sweatpants** – charcoal (4) | Warm, flexible, perfect for an active look. |
| Shoes| **Boots (waterproof)** – brown (5) | Keeps feet dry and warm; sturdy for any outdoor activity. |
| Accessories| **Beanie** – charcoal (5) <br> **Gloves (winter insulated)** – navy (4) <br> **Wool scarf** – dark gray (4) | Essential cold‑weather protection while staying simple and sporty. |

**Quick look:**  
Thermal tee → heavyweight hoodie → wool sweater → olive puffer → charcoal sweatpants → brown waterproof boots, finished with a charcoal beanie, navy insulated gloves, and a dark‑gray wool scarf.

All pieces are in your inventory and carry the highest preference scores, giving you warmth and a clean, sporty aesthetic for tomorrow’s chilly, clear day. Enjoy!

In [11]:
out = await run(style="elegant", time="today")
mdprint(out.final_output)

[11/20/25 12:33:30] INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"         ]8;id=590880;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=336293;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     Received session ID: 4e713bd0aa3347ab944da3df12e75b26           ]8;id=209451;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=528072;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py#134\134]8;;\

                    INFO     Negotiated protocol version: 2025-06-18                         ]8;id=971274;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py\streamable_http.py]8;;\:]8;id=269063;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/mcp/client/streamable_http.py#146\146]8;;\

[11/20/25 12:33:31] INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 202 Accepted"   ]8;id=202722;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=724636;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

                    INFO     HTTP Request: GET http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"          ]8;id=446801;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=315871;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

...

                    INFO     HTTP Request: POST http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"         ]8;id=866359;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=591252;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

[11/20/25 12:33:39] WARNING  OPENAI_API_KEY is not set, skipping trace export                      ]8;id=509143;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/agents/tracing/processors.py\processors.py]8;;\:]8;id=891468;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/agents/tracing/processors.py#96\96]8;;\

[11/20/25 12:33:40] INFO     HTTP Request: POST                                                     ]8;id=956224;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=493053;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\
                             https://api.helmholtz-blablador.fz-juelich.de/v1/chat/completions                     
                             "HTTP/1.1 200 OK"                                                                     

[11/20/25 12:33:41] INFO     HTTP Request: DELETE http://127.0.0.1:8000/mcp "HTTP/1.1 200 OK"       ]8;id=970909;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=226496;file:///Users/gienapp/Gits/teaching-ks-ws25-agentic-ai/.venv/lib/python3.10/site-packages/httpx/_client.py#1740\1740]8;;\

**Elegant‑Winter Outfit (Leipzig 20 Nov, ~2 °C, snow‑risk)**  

- **Base layer:** thermal base layer – black (rating 4)  
- **Mid‑layer:** sweater (thick wool) – oatmeal (rating 5)  
- **Shirt:** dress shirt – light‑blue (rating 2)  
- **Outer‑layer:** **blazer** – midnight blue (rating 2) **under** a **winter coat** – forest green (rating 4) – gives the polished look while keeping you warm.  
- **Bottom:** dress pants – navy (rating 2)  
- **Footwear:** boots (winter insulated) – black (rating 4) – suitable for snow and still dressy.  
- **Accessories:**  
  * scarf (wool) – dark gray (rating 4)  
  * gloves (winter insulated) – navy (rating 4)  
  * beanie – charcoal (rating 5) (optional, can be tucked under the coat collar)  
  * umbrella – black (rating 5) for any snowfall.

**Why it works:**  
- Layers (thermal + thick wool + blazer) give ≥ 10 °C effective warmth, offsetting the 2 °C ambient temperature and wind.  
- The blazer maintains an elegant silhouette; the coat adds weather protection without sacrificing style.  
- Dark‑toned dress pants and black insulated boots keep the look refined yet practical for snow.  
- Wool scarf and insulated gloves complete the ensemble while matching the sophisticated palette.  